# Prerequisites
Please run the prerequiste cells before trying the examples.

## Install python module dependencies

In [ ]:
import sys
def installModule(projectName:str, moduleName:str=None):
    '''Installs and loads the given module if not already installed'''
    if moduleName is None:
        moduleName=projectName
    !python -m pip install -U --no-input $projectName
    print(f'{projectName} installed')
    %reload_ext $moduleName

installModule('jupyter-xml')
installModule('jupyter-rdfify==1.0.1','jupyter-rdfify')
installModule('SPARQLWrapper')
installModule('tabulate')
installModule("pylodstorage", "lodstorage")

.# Querying wikidata
[wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page) provides an [SPARQL endpoint](https://query.wikidata.org/) for querying their triplestore. 

To get a response from the endpoint the query is sent as post request and returns the results in __json__ or __xml__ format.

The result is then post-processed into a more usable format. Depending on the post-processing the results are converted into python types and this step can also include a merging of results. For example if a proeprty has multiple values and all match the query each value will result in its own row (For more details in the result reduction see [SPARQL Transformer](https://www.eurecom.fr/en/publication/5927/download/data-publi-5927.pdf)).

[![](https://mermaid.ink/img/eyJjb2RlIjoiZmxvd2NoYXJ0IExSXG4gICAgQShRdWVyeSkgLS0-fHBvc3QgcmVxdWVzdHwgQihldmFsdWF0ZSBRdWVyeSlcbiAgICBCIC0tPnxyZXNwb25zZXwgQyhKU09OIFJlc3BvbnNlKVxuICAgIEMgLS0-fHBvc3QgcHJvY2Vzc3wgRChcIkxpc3Qgb2YgRGljdHMgKExvRClcIikiLCJtZXJtYWlkIjp7InRoZW1lIjoiZGVmYXVsdCJ9LCJ1cGRhdGVFZGl0b3IiOmZhbHNlLCJhdXRvU3luYyI6dHJ1ZSwidXBkYXRlRGlhZ3JhbSI6ZmFsc2V9)](https://mermaid.live/edit/#eyJjb2RlIjoiZmxvd2NoYXJ0IExSXG4gICAgQShRdWVyeSkgLS0-fHBvc3QgcmVxdWVzdHwgQihldmFsdWF0ZSBRdWVyeSlcbiAgICBCIC0tPnxyZXNwb25zZXwgQyhKU09OIFJlc3BvbnNlKVxuICAgIEMgLS0-fHBvc3QgcHJvY2Vzc3wgRChcIkxpc3Qgb2YgRGljdHMgKExvRClcIikiLCJtZXJtYWlkIjoie1xuICBcInRoZW1lXCI6IFwiZGVmYXVsdFwiXG59IiwidXBkYXRlRWRpdG9yIjpmYWxzZSwiYXV0b1N5bmMiOnRydWUsInVwZGF0ZURpYWdyYW0iOmZhbHNlfQ)


[![](https://mermaid.ink/img/eyJjb2RlIjoic2VxdWVuY2VEaWFncmFtIFxuICAgIHBhcnRpY2lwYW50IEEgYXMgQ2xpZW50XG4gICAgcGFydGljaXBhbnQgQiBhcyBTUEFSUUwgRW5kcG9pbnRcbiAgICBwYXJ0aWNpcGFudCBDIGFzIFRyaXBsZXN0b3JlXG4gICAgYWN0aXZhdGUgQVxuICAgIEEtPj5COiBwb3N0IFF1ZXJ5XG4gICAgYWN0aXZhdGUgQlxuICAgIGxvb3AgUXVlcnkgRXZhbHVhdGlvblxuICAgICAgICBhY3RpdmF0ZSBDXG4gICAgICAgIEItLT4-QzogXG4gICAgICAgIG5vdGUgcmlnaHQgb2YgQzogRmluZCBtYXRjaGluZyB0cmlwbGVzXG4gICAgICAgIEMtLT4-QjogXG4gICAgICAgIGRlYWN0aXZhdGUgQ1xuICAgIGVuZFxuICAgIEItLT4-QTogSlNPTi9YTUwgUmVzcG9uc2VcbiAgICBkZWFjdGl2YXRlIEJcbiAgICBBLS0-PkE6IHBvc3QtcHJvY2VzcyByZXN1bHRzXG4gICAgZGVhY3RpdmF0ZSBBIiwibWVybWFpZCI6eyJ0aGVtZSI6ImRlZmF1bHQifSwidXBkYXRlRWRpdG9yIjpmYWxzZSwiYXV0b1N5bmMiOnRydWUsInVwZGF0ZURpYWdyYW0iOmZhbHNlfQ)](https://mermaid.live/edit/#eyJjb2RlIjoic2VxdWVuY2VEaWFncmFtIFxuICAgIHBhcnRpY2lwYW50IEEgYXMgQ2xpZW50XG4gICAgcGFydGljaXBhbnQgQiBhcyBTUEFSUUwgRW5kcG9pbnRcbiAgICBwYXJ0aWNpcGFudCBDIGFzIFRyaXBsZXN0b3JlXG4gICAgYWN0aXZhdGUgQVxuICAgIEEtPj5COiBwb3N0IFF1ZXJ5XG4gICAgYWN0aXZhdGUgQlxuICAgIGxvb3AgUXVlcnkgRXZhbHVhdGlvblxuICAgICAgICBhY3RpdmF0ZSBDXG4gICAgICAgIEItLT4-QzogXG4gICAgICAgIG5vdGUgcmlnaHQgb2YgQzogRmluZCBtYXRjaGluZyB0cmlwbGVzXG4gICAgICAgIEMtLT4-QjogXG4gICAgICAgIGRlYWN0aXZhdGUgQ1xuICAgIGVuZFxuICAgIEItLT4-QTogSlNPTi9YTUwgUmVzcG9uc2VcbiAgICBkZWFjdGl2YXRlIEJcbiAgICBBLS0-PkE6IHBvc3QtcHJvY2VzcyByZXN1bHRzXG4gICAgZGVhY3RpdmF0ZSBBIiwibWVybWFpZCI6IntcbiAgXCJ0aGVtZVwiOiBcImRlZmF1bHRcIlxufSIsInVwZGF0ZUVkaXRvciI6ZmFsc2UsImF1dG9TeW5jIjp0cnVlLCJ1cGRhdGVEaWFncmFtIjpmYWxzZX0)

## Example
Query all __James Bond__ movies
### Sending the Query as posts request

In [ ]:
import requests

url = "https://query.wikidata.org/sparql"
query="""#James Bond Movies
SELECT ?item ?movieTitle (MIN(?date) AS ?firstReleased)
WHERE {
  ?item wdt:P179 wd:Q2484680;
        rdfs:label ?movieTitle;
        wdt:P577 ?date.
  FILTER(lang(?movieTitle)="en")
} GROUP BY ?item ?movieTitle
ORDER BY DESC(?firstReleased)
LIMIT 3
"""
params={
    "query":query,
    "format":"xml" # or json
}
response = requests.request("POST", url, params=params)

print(response.text)

### Post-Process the Response

In [ ]:
from xml.etree import ElementTree
from tabulate import tabulate
tree=ElementTree.fromstring(response.text)
prefix_map={"res":"http://www.w3.org/2005/sparql-results#"}
lod=[]
for result in tree.findall(".//res:results/res:result", prefix_map):
    record={}
    for binding in result.findall("./res:binding", prefix_map):
        name=binding.attrib['name']
        value=binding.find("./*").text
        record[name]=value
    lod.append(record)

# pretty print the list of dicts (LoD)
print(tabulate(lod, headers="keys"))

# SPARQL Libraries
To simplify the querying process there existdifferent python libraries that take care of the post-processing of the result and directly return the result in  python types. Such Libraries are:

|Library| Description|
|---|---|
|RDFify| SPARQL support for jupyter notebooks with the use of jupyter magicwords (based on SPARQLWrapper)|
|SPARQLWrapper| |
|pyLODStorage| supports vonversion of the results into other storage formats and provides a documentation of the executed query (based on SPARQLWrapper)|

In the following the mentioned libraries are used to execute and printout the result of the following query:

```sparql
SELECT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
  ?substance wdt:P31 wd:Q11173.
  ?substance wdt:P231 ?CAS.
  ?substance wdt:P274 ?formula.
  ?substance wdt:P117  ?structure.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 15
```
> see [pyLodStorage Random Substances with CAS number example](http://wiki.bitplan.com/index.php/PyLoDStorage#15_Random_substances_with_CAS_number)

## RDFify
### RDFify tool documentation

In [ ]:
%rdf turtle -h

In [ ]:
%rdf sparql -h

In [ ]:
%rdf --help

### RDFify Example Usage

In [ ]:
%%rdf sparql --endpoint https://query.wikidata.org/sparql -d table -f json -s substances
SELECT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
  ?substance wdt:P31 wd:Q11173.
  ?substance wdt:P231 ?CAS.
  ?substance wdt:P274 ?formula.
  ?substance wdt:P117  ?structure.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 15


In [ ]:
store = %rdf -r
g = store["rdfresults"]
print(g['substances']) # SPARQLWrapper QueryResult

## [SPARQLWrapper](https://sparqlwrapper.readthedocs.io/en/latest/main.html)
SPARQLWrapper result Format:
* head
  * vars (list of colimn names)
* results
  * bindings (list with all row values)
    * <column name>
       * type
       * value
    
An SPARWLWrapper result can be converted to an list of dicts (LoD) with the following list comprehentions:
```python
lod=[{column:cell["value"] for column, cell in row.items()} for row in results["results"]["bindings"]] 
```

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
from tabulate import tabulate
from IPython.display import Image, SVG
# List of Dicts (LoD) 

q = """
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
SELECT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
  ?substance wdt:P31 wd:Q11173.
  ?substance wdt:P231 ?CAS.
  ?substance wdt:P274 ?formula.
  ?substance wdt:P117  ?structure.
}
LIMIT 15
"""

sparql = SPARQLWrapper("https://qlever.cs.uni-freiburg.de/api/wikidata")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results)
# print raw results
#print(results)
lod=[{column:cell["value"] for column, cell in row.items()} for row in results["results"]["bindings"]]
print(tabulate(lod, headers="keys"))


## pyLODStorage

pyLODStorage directly converts the query result into an list of dicts (LoD) and also provides methods to store the LoD in different formats such as JSON, SQL and CSV.
This library also supports the use of named queries which alow to store metadata of the query such as a name, title, description and language.

An additional advantage is that it provides a pretty print of the query and result in the jupyter environment. 

In [ ]:
from IPython.display import display, Markdown
from lodstorage.query import QueryManager, Query
from lodstorage.sparql import SPARQL
import copy

endpoint=SPARQL("https://query.wikidata.org/sparql")

queryStr="""# List of 15 random chemical components with CAS-Number, formula and structure
# see also https://github.com/WolfgangFahl/pyLoDStorage/issues/46
# WF 2021-08-23
SELECT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
?substance wdt:P31 wd:Q11173.
?substance wdt:P231 ?CAS.
?substance wdt:P274 ?formula.
?substance wdt:P117  ?structure.
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 15
"""
query=Query(
    query=queryStr, 
    name="CAS15", 
    title="15 Random substances with CAS number", 
    lang="sparql",
    description="Wikidata SPARQL query showing the 15 random chemical substances with their CAS Number"
)
try:
    qlod=endpoint.queryAsListOfDicts(query.query)
    lod=copy.deepcopy(qlod)
    
    # pretty print the query and its result
    doc=query.documentQueryResult(lod, tablefmt="github",floatfmt=".0f",tryItUrl="https://query.wikidata.org")
    display(Markdown(str(doc)))
except Exception as ex:
            print(f"{query.title} at {endpoint.sparql.endpoint} failed: {ex}")